<p align="center">
<img src="Images/sorbonne_logo.png" alt="Logo" width="300"/>
</p>

# **PANEL DATA ECONOMETRICS - Code File Part 2**

* **Author**: Elia Landini
* **Student ID**: 12310239
* **Course**: EESM2-Financial Economics 
* **Class**: Panel Data Econometrics
* **Supervisor**: Jean-Bernard Chatelain 
* **Reference Paper**: Lofaro, A., & Di Bucchianico, S. (2025). Impact of monetary policy on functional income distribution: A panel vector autoregressive analysis. Economic Modelling, 107227
* **Reference Repository**: https://github.com/EliaLand/PVAR_MonetaryPolicy_FunctionalIncome

### **0) REQUIREMENTS SET-UP**

In [2]:
# Requirements.txt file installation
# !pip install -r requirements.txt

In [16]:
# Libraries import
import warnings
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%matplotlib inline
import seaborn as sns
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import levene
from scipy.stats import ks_2samp
from scipy.stats import kstest
from scipy.stats import pearsonr
from scipy.stats import jarque_bera
import statsmodels.api as sm
from linearmodels.panel import PanelOLS, RandomEffects
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.nonparametric.smoothers_lowess import lowess
from linearmodels.panel import compare
import plotly.express as px
from IPython.core.display import HTML
from IPython.display import Image

In [4]:
# Statistical Significance labelling 
def significance_stars(p):
    if p < 0.001:
        return "***"  
    elif p < 0.01:
        return "**"    
    elif p < 0.05:
        return "*"   
    else:
        return ""

In [5]:
# We supress potential warnings with this command
warnings.filterwarnings("ignore")

In [11]:
# Data loading from CodeFile Part 1 for raw data and transformed variables
final_trans_df = pd.read_csv("Data/final_trans_df.csv")
final_trans_df["year"] = pd.to_datetime(final_trans_df["year"])
# (!!!) We need to ensure a 2-level multi-index (or sm estimators won't work, in our case i="country" and t="year")
final_trans_df = final_trans_df.set_index(["country", "year"])
final_trans_df

i      P          W         WR           GDP  \
country   year                                                               
Australia 1970-01-01   7.108333    9.1   6.413648  76.120158  2.835193e+05   
          1971-01-01   7.008333    9.6   6.909308  77.071751  2.946057e+05   
          1972-01-01   5.083333   10.2   7.284607  75.808223  3.023225e+05   
          1973-01-01   6.983333   11.1   8.096658  74.465333  3.147389e+05   
          1974-01-01  13.158333   12.9   9.688749  75.817692  3.189447e+05   
...                         ...    ...        ...        ...           ...   
USA       2015-01-01   0.226667  100.0  65.600000  62.655205  1.820602e+07   
          2016-01-01   0.644167  101.3  66.400000  62.819300  1.850960e+07   
          2017-01-01   1.152500  103.4  68.600000  63.695450  1.892457e+07   
          2018-01-01   2.188333  105.9  70.900000  64.279238  1.948198e+07   
          2019-01-01   2.207500  107.9  73.100000  65.093500  1.992897e+07   

                        LS       PCOM        UN   SHORTUN  LONGUN  ...  \
country   year                                                     ...   
Australia 1970-01-01  59.7   2.113115  1.653364       NaN     NaN  ...   
          1971-01-01  60.4   2.709369  1.907799       NaN     NaN  ...   
          1972-01-01  60.3   2.925285  2.610752       NaN     NaN  ...   
          1973-01-01  60.8   4.174010  2.309362       NaN     NaN  ...   
          1974-01-01  66.9  13.802962  2.670259       NaN     NaN  ...   
...                    ...        ...       ...       ...     ...  ...   
USA       2015-01-01  56.8  66.099106  5.279920  5968.333  2328.0  ...   
          2016-01-01  56.8  56.023884  4.869110  5746.000  2005.0  ...   
          2017-01-01  56.9  69.357989  4.355201  5294.250  1688.0  ...   
          2018-01-01  56.8  89.358138  3.895676  4964.917  1349.0  ...   
          2019-01-01  56.9  78.251714  3.669213  4733.583  1267.0  ...   

                       WR_TWFE      GDP_TWFE   LS_TWFE     PCOM_TWFE  \
country   year                                                         
Australia 1970-01-01  6.652899  6.417193e+05 -1.599571  1.421085e-14   
          1971-01-01  6.815177  6.194218e+05 -1.535286  2.131628e-14   
          1972-01-01  5.360967  5.765079e+05 -1.728143  0.000000e+00   
          1973-01-01  0.878244  5.265239e+05 -0.949571  0.000000e+00   
          1974-01-01  2.643711  5.213183e+05  3.743286  1.421085e-14   
...                        ...           ...       ...           ...   
USA       2015-01-01  9.355536  5.625355e+06  0.469433  0.000000e+00   
          2016-01-01  8.342223  5.882897e+06  0.702767  1.776357e-15   
          2017-01-01  8.851479  6.232285e+06  1.196100 -1.065814e-14   
          2018-01-01  8.792271  6.728788e+06  0.876100 -1.421085e-14   
          2019-01-01  9.679027  7.128463e+06  0.522767 -1.421085e-14   

                       UN_TWFE  SHORTUN_TWFE  LONGUN_TWFE       LF_TWFE  \
country   year                                                            
Australia 1970-01-01  0.083200           NaN          NaN   3746.924114   
          1971-01-01 -0.010331           NaN          NaN   3673.600806   
          1972-01-01  0.306079           NaN          NaN   3516.775114   
          1973-01-01  0.280121           NaN          NaN   3252.168575   
          1974-01-01  0.703663           NaN          NaN   2769.733122   
...                        ...           ...          ...           ...   
USA       2015-01-01 -2.158072   -366.715087   510.316135  23419.420004   
          2016-01-01 -2.012960   -557.504449   282.317657  25183.856004   
          2017-01-01 -1.792373   -943.479016    65.509990  26116.245337   
          2018-01-01 -1.493159  -1224.491436  -171.593610  27580.072337   
          2019-01-01 -1.360396  -1436.126789  -192.683076  28785.793337   

                      REER_TWFE   SH_TWFE  
country   year                             
Australia 1970-01-01   2.873095 -2.544304  
       

### **2) PART 2A - CLASSIC BENCHMARK MULTIVARIATE PANEL DATA ESTIMATORS**

##### <span style="color: orange"> **2.20) In a single table, report and comment the results of estimations of Between, Within (one-way fixed effects, (fe)) and Mundlak (random effects (re) including all X(i.) as regressors), two-way fixed effects (add year dummies in fe regression) and First differences, including all explanatory variables except the ones with high near-multicollinearity after their transformation.** </span>

In [ ]:
# Model 1 - Estimation Results 
# Model 1 entails the state space vector for the PVAR model adopted by Lofaro, A., & Di Bucchianico, S. (2025).
# The vector is so defined: y = [PCOM, GDP, UN, P, WR, LS, i]
# Assumed Dependent Y = GDP, Assumed X = [UN, P, WR, LS, i]
# (!!!) PCOM must be taken out as it causes TWFE to crash as all the between-countries variability has been absorbed already (=0)

# //////
# Set-Up
# //////
df = final_trans_df.copy()
# Variable definition 
X_vars = ["UN", "P", "WR", "LS", "i"] 
y_var = "GDP"

# /////////////////
# Between Estimator
# /////////////////
# (!!!) Both entity (i) and time effects (t) off
# (!!!) Given the heterogeneity of our panel,w e deploy robust covariance
between_mod = PanelOLS(
    df[y_var],
    sm.add_constant(df[X_vars]),
    entity_effects=False,
    time_effects=False
).fit(cov_type="robust")

# /////////////////////
# OWFE-Within Estimator
# /////////////////////
# (!!!) Here we turn on entity effects (i)
within_mod = PanelOLS(
    df[y_var],
    df[X_vars],
    entity_effects=True
).fit(cov_type="robust")

# ///////////////////////////////
# Mundlak (RE + group means of X)
# ///////////////////////////////
# Iteration over regressors to get the respective group means within each country (level=0)
# (!!!) We no longer refer to "country" as a column, but as a index level (in this case the first, so 0)
for var in X_vars:
    df[f"{var}_mean"] = df.groupby(level=0)[var].transform("mean")
mundlak_X = X_vars + [f"{var}_mean" for var in X_vars]
# RE a la Mundlak
mundlak_mod = RandomEffects(
    df[y_var],
    sm.add_constant(df[mundlak_X])
).fit(cov_type="robust")

# /////////////////////
# TWFE-Within Estimator
# /////////////////////
# (!!!) Both entity (i) and time effects (t) on this time 
# (!!!) Same approach as between and within estimators, but we turn on both parameters this time
TWFE_mod = PanelOLS(
    df[y_var],
    df[X_vars],
    entity_effects=True,
    time_effects=True
).fit(cov_type="robust")

# ///////////////////////////
# First Differences Estimator
# ///////////////////////////
# We have to take NaN out, otherwise the run crashes
# (!!!) Faster to go directly with .diff() instead of filtering the already transformed variables from final_trans_df
# (!!!) I cannot call the df starting with a number like 1diff, like I would, cuz it's not accepted 
diff_df = df.groupby(level=0).diff().dropna()
# We now plug as usual 1diff in the PanelOLS
diff_mod = PanelOLS(
    diff_df[y_var],
    diff_df[X_vars]
).fit(cov_type="robust")

# ////////////
# Tabular Form
# ////////////
# (!!!) Nice comparison command (FANTASTIC!), it basically requires no adjustments
results = {
    "Between": between_mod,
    "OWFE-Within": within_mod,
    "RE-Mundlak": mundlak_mod,
    "TWFE-Within": TWFE_mod,
    "First Differences": diff_mod
}

comparison = compare(results)
comparison

,Between,OWFE-Within,RE-Mundlak,TWFE-Within,First Differences
Dep. Variable,GDP,GDP,GDP,GDP,GDP
Estimator,PanelOLS,PanelOLS,RandomEffects,PanelOLS,PanelOLS
No. Observations,731,731,731,731,555
Cov. Est.,Robust,Robust,Robust,Robust,Robust
R-squared,0.1035,0.2890,0.2891,0.0727,0.2673
R-Squared (Within),0.1498,0.2890,0.2890,0.0705,0.1728
R-Squared (Between),0.1005,0.3205,0.2969,0.2776,0.3076
R-Squared (Overall),0.1035,0.3093,0.3076,0.2471,0.2673
F-statistic,16.748,57.793,29.286,10.378,40.128
P-value (F-stat),0.0000,0.0000,0.0000,0.0000,0.0000


In [18]:
# Model 2 - Estimation Results 
# Model 2 entails the state space vector for the PVAR model adopted by Lofaro, A., & Di Bucchianico, S. (2025).
# The vector is so defined: y = [PCOM, GDP, UN, P, W, LS, i]
# Assumed Dependent Y = GDP, Assumed X = [UN, P, W, LS, i]
# (!!!) PCOM must be taken out as it causes TWFE to crash as all the between-countries variability has been absorbed already (=0)

# //////
# Set-Up
# //////
df = final_trans_df.copy()
# Variable definition 
# (!!!) Nominal compensation (W), instead of real (WR)
X_vars = ["UN", "P", "W", "LS", "i"] 
y_var = "GDP"

# /////////////////
# Between Estimator
# /////////////////
# (!!!) Both entity (i) and time effects (t) off
# (!!!) Given the heterogeneity of our panel,w e deploy robust covariance
between_mod = PanelOLS(
    df[y_var],
    sm.add_constant(df[X_vars]),
    entity_effects=False,
    time_effects=False
).fit(cov_type="robust")

# /////////////////////
# OWFE-Within Estimator
# /////////////////////
# (!!!) Here we turn on entity effects (i)
within_mod = PanelOLS(
    df[y_var],
    df[X_vars],
    entity_effects=True
).fit(cov_type="robust")

# ///////////////////////////////
# Mundlak (RE + group means of X)
# ///////////////////////////////
# Iteration over regressors to get the respective group means within each country (level=0)
# (!!!) We no longer refer to "country" as a column, but as a index level (in this case the first, so 0)
for var in X_vars:
    df[f"{var}_mean"] = df.groupby(level=0)[var].transform("mean")
mundlak_X = X_vars + [f"{var}_mean" for var in X_vars]
# RE a la Mundlak
mundlak_mod = RandomEffects(
    df[y_var],
    sm.add_constant(df[mundlak_X])
).fit(cov_type="robust")

# /////////////////////
# TWFE-Within Estimator
# /////////////////////
# (!!!) Both entity (i) and time effects (t) on this time 
# (!!!) Same approach as between and within estimators, but we turn on both parameters this time
TWFE_mod = PanelOLS(
    df[y_var],
    df[X_vars],
    entity_effects=True,
    time_effects=True
).fit(cov_type="robust")

# ///////////////////////////
# First Differences Estimator
# ///////////////////////////
# We have to take NaN out, otherwise the run crashes
# (!!!) Faster to go directly with .diff() instead of filtering the already transformed variables from final_trans_df
# (!!!) I cannot call the df starting with a number like 1diff, like I would, cuz it's not accepted 
diff_df = df.groupby(level=0).diff().dropna()
# We now plug as usual 1diff in the PanelOLS
diff_mod = PanelOLS(
    diff_df[y_var],
    diff_df[X_vars]
).fit(cov_type="robust")

# ////////////
# Tabular Form
# ////////////
# (!!!) Nice comparison command (FANTASTIC!)
results = {
    "Between": between_mod,
    "OWFE-Within": within_mod,
    "RE-Mundlak": mundlak_mod,
    "TWFE-Within": TWFE_mod,
    "First Differences": diff_mod
}

comparison = compare(results)
comparison

,Between,OWFE-Within,RE-Mundlak,TWFE-Within,First Differences
Dep. Variable,GDP,GDP,GDP,GDP,GDP
Estimator,PanelOLS,PanelOLS,RandomEffects,PanelOLS,PanelOLS
No. Observations,731,731,731,731,555
Cov. Est.,Robust,Robust,Robust,Robust,Robust
R-squared,0.1654,0.3257,0.3259,0.1510,0.2534
R-Squared (Within),0.1355,0.3257,0.3257,-0.4487,0.1698
R-Squared (Between),0.1708,0.3712,0.3343,0.1900,0.2770
R-Squared (Overall),0.1654,0.3616,0.3453,0.1103,0.2534
F-statistic,28.744,68.697,34.809,23.555,37.332
P-value (F-stat),0.0000,0.0000,0.0000,0.0000,0.0000


In [ ]:
# Model 3 - Estimation Results 
# Model 3 entails the state space vector for the PVAR model adopted by Lofaro, A., & Di Bucchianico, S. (2025).
# The vector is so defined: y = [PCOM, GDP, SHORTUN, P, WR, LS, i]
# Assumed Dependent Y = GDP, Assumed X = [SHORTUN, P, WR, LS, i]
# (!!!) PCOM must be taken out as it causes TWFE to crash as all the between-countries variability has been absorbed already (=0)

# //////
# Set-Up
# //////
df = final_trans_df.copy()
# Variable definition 
# (!!!) Short-run Unemployemnt (SHORTUN), instead of annual unemployment rate (UN), and WR back in instead of W
X_vars = ["SHORTUN", "P", "WR", "LS", "i"] 
y_var = "GDP"

# /////////////////
# Between Estimator
# /////////////////
# (!!!) Both entity (i) and time effects (t) off
# (!!!) Given the heterogeneity of our panel,w e deploy robust covariance
between_mod = PanelOLS(
    df[y_var],
    sm.add_constant(df[X_vars]),
    entity_effects=False,
    time_effects=False
).fit(cov_type="robust")

# /////////////////////
# OWFE-Within Estimator
# /////////////////////
# (!!!) Here we turn on entity effects (i)
within_mod = PanelOLS(
    df[y_var],
    df[X_vars],
    entity_effects=True
).fit(cov_type="robust")

# ///////////////////////////////
# Mundlak (RE + group means of X)
# ///////////////////////////////
# Iteration over regressors to get the respective group means within each country (level=0)
# (!!!) We no longer refer to "country" as a column, but as a index level (in this case the first, so 0)
for var in X_vars:
    df[f"{var}_mean"] = df.groupby(level=0)[var].transform("mean")
mundlak_X = X_vars + [f"{var}_mean" for var in X_vars]
# RE a la Mundlak
mundlak_mod = RandomEffects(
    df[y_var],
    sm.add_constant(df[mundlak_X])
).fit(cov_type="robust")

# /////////////////////
# TWFE-Within Estimator
# /////////////////////
# (!!!) Both entity (i) and time effects (t) on this time 
# (!!!) Same approach as between and within estimators, but we turn on both parameters this time
TWFE_mod = PanelOLS(
    df[y_var],
    df[X_vars],
    entity_effects=True,
    time_effects=True
).fit(cov_type="robust")

# ///////////////////////////
# First Differences Estimator
# ///////////////////////////
# We have to take NaN out, otherwise the run crashes
# (!!!) Faster to go directly with .diff() instead of filtering the already transformed variables from final_trans_df
# (!!!) I cannot call the df starting with a number like 1diff, like I would, cuz it's not accepted 
diff_df = df.groupby(level=0).diff().dropna()
# We now plug as usual 1diff in the PanelOLS
diff_mod = PanelOLS(
    diff_df[y_var],
    diff_df[X_vars]
).fit(cov_type="robust")

# ////////////
# Tabular Form
# ////////////
# (!!!) Nice comparison command (FANTASTIC!)
results = {
    "Between": between_mod,
    "OWFE-Within": within_mod,
    "RE-Mundlak": mundlak_mod,
    "TWFE-Within": TWFE_mod,
    "First Differences": diff_mod
}

comparison = compare(results)
comparison

,Between,OWFE-Within,RE-Mundlak,TWFE-Within,First Differences
Dep. Variable,GDP,GDP,GDP,GDP,GDP
Estimator,PanelOLS,PanelOLS,RandomEffects,PanelOLS,PanelOLS
No. Observations,585,585,585,585,555
Cov. Est.,Robust,Robust,Robust,Robust,Robust
R-squared,0.8222,0.4168,0.5705,0.2520,0.3935
R-Squared (Within),0.1165,0.4168,0.4154,-1.6040,0.5530
R-Squared (Between),0.9466,-1.8818,0.9495,-8.0634,0.3036
R-Squared (Overall),0.8222,-1.1028,0.8756,-5.5667,0.3935
F-statistic,535.50,80.751,76.257,34.760,71.376
P-value (F-stat),0.0000,0.0000,0.0000,0.0000,0.0000


In [21]:
# Model 4 - Estimation Results 
# Model 4 entails the state space vector for the PVAR model adopted by Lofaro, A., & Di Bucchianico, S. (2025).
# The vector is so defined: y = [PCOM, GDP, LONGUN, P, WR, LS, i]
# Assumed Dependent Y = GDP, Assumed X = [LONGUN, P, WR, LS, i]
# (!!!) PCOM must be taken out as it causes TWFE to crash as all the between-countries variability has been absorbed already (=0)

# //////
# Set-Up
# //////
df = final_trans_df.copy()
# Variable definition 
# (!!!) Long-run Unemployemnt (LONGUN), instead of annual unemployment rate (UN)
X_vars = ["LONGUN", "P", "WR", "LS", "i"] 
y_var = "GDP"

# /////////////////
# Between Estimator
# /////////////////
# (!!!) Both entity (i) and time effects (t) off
# (!!!) Given the heterogeneity of our panel,w e deploy robust covariance
between_mod = PanelOLS(
    df[y_var],
    sm.add_constant(df[X_vars]),
    entity_effects=False,
    time_effects=False
).fit(cov_type="robust")

# /////////////////////
# OWFE-Within Estimator
# /////////////////////
# (!!!) Here we turn on entity effects (i)
within_mod = PanelOLS(
    df[y_var],
    df[X_vars],
    entity_effects=True
).fit(cov_type="robust")

# ///////////////////////////////
# Mundlak (RE + group means of X)
# ///////////////////////////////
# Iteration over regressors to get the respective group means within each country (level=0)
# (!!!) We no longer refer to "country" as a column, but as a index level (in this case the first, so 0)
for var in X_vars:
    df[f"{var}_mean"] = df.groupby(level=0)[var].transform("mean")
mundlak_X = X_vars + [f"{var}_mean" for var in X_vars]
# RE a la Mundlak
mundlak_mod = RandomEffects(
    df[y_var],
    sm.add_constant(df[mundlak_X])
).fit(cov_type="robust")

# /////////////////////
# TWFE-Within Estimator
# /////////////////////
# (!!!) Both entity (i) and time effects (t) on this time 
# (!!!) Same approach as between and within estimators, but we turn on both parameters this time
TWFE_mod = PanelOLS(
    df[y_var],
    df[X_vars],
    entity_effects=True,
    time_effects=True
).fit(cov_type="robust")

# ///////////////////////////
# First Differences Estimator
# ///////////////////////////
# We have to take NaN out, otherwise the run crashes
# (!!!) Faster to go directly with .diff() instead of filtering the already transformed variables from final_trans_df
# (!!!) I cannot call the df starting with a number like 1diff, like I would, cuz it's not accepted 
diff_df = df.groupby(level=0).diff().dropna()
# We now plug as usual 1diff in the PanelOLS
diff_mod = PanelOLS(
    diff_df[y_var],
    diff_df[X_vars]
).fit(cov_type="robust")

# ////////////
# Tabular Form
# ////////////
# (!!!) Nice comparison command (FANTASTIC!)
results = {
    "Between": between_mod,
    "OWFE-Within": within_mod,
    "RE-Mundlak": mundlak_mod,
    "TWFE-Within": TWFE_mod,
    "First Differences": diff_mod
}

comparison = compare(results)
comparison

,Between,OWFE-Within,RE-Mundlak,TWFE-Within,First Differences
Dep. Variable,GDP,GDP,GDP,GDP,GDP
Estimator,PanelOLS,PanelOLS,RandomEffects,PanelOLS,PanelOLS
No. Observations,585,585,585,585,555
Cov. Est.,Robust,Robust,Robust,Robust,Robust
R-squared,0.3218,0.4761,0.4727,0.3055,0.3017
R-Squared (Within),0.0681,0.4761,0.4761,-0.5952,0.2922
R-Squared (Between),0.3554,-3.2211,0.4187,-10.572,0.3005
R-Squared (Overall),0.3218,-2.0440,0.4601,-7.3045,0.3017
F-statistic,54.936,102.70,51.455,45.396,47.526
P-value (F-stat),0.0000,0.0000,0.0000,0.0000,0.0000


In [22]:
# Model 5 - Estimation Results 
# Model 5 entails the state space vector for the PVAR model adopted by Lofaro, A., & Di Bucchianico, S. (2025).
# The vector is so defined: y = [PCOM, GDP, LF, P, WR, LS, i]
# Assumed Dependent Y = GDP, Assumed X = [LF, P, WR, LS, i]
# (!!!) PCOM must be taken out as it causes TWFE to crash as all the between-countries variability has been absorbed already (=0)

# //////
# Set-Up
# //////
df = final_trans_df.copy()
# Variable definition 
# (!!!) Labour Force Share (LS), instead of annual unemployment rate (UN)
X_vars = ["LF", "P", "WR", "LS", "i"] 
y_var = "GDP"

# /////////////////
# Between Estimator
# /////////////////
# (!!!) Both entity (i) and time effects (t) off
# (!!!) Given the heterogeneity of our panel,w e deploy robust covariance
between_mod = PanelOLS(
    df[y_var],
    sm.add_constant(df[X_vars]),
    entity_effects=False,
    time_effects=False
).fit(cov_type="robust")

# /////////////////////
# OWFE-Within Estimator
# /////////////////////
# (!!!) Here we turn on entity effects (i)
within_mod = PanelOLS(
    df[y_var],
    df[X_vars],
    entity_effects=True
).fit(cov_type="robust")

# ///////////////////////////////
# Mundlak (RE + group means of X)
# ///////////////////////////////
# Iteration over regressors to get the respective group means within each country (level=0)
# (!!!) We no longer refer to "country" as a column, but as a index level (in this case the first, so 0)
for var in X_vars:
    df[f"{var}_mean"] = df.groupby(level=0)[var].transform("mean")
mundlak_X = X_vars + [f"{var}_mean" for var in X_vars]
# RE a la Mundlak
mundlak_mod = RandomEffects(
    df[y_var],
    sm.add_constant(df[mundlak_X])
).fit(cov_type="robust")

# /////////////////////
# TWFE-Within Estimator
# /////////////////////
# (!!!) Both entity (i) and time effects (t) on this time 
# (!!!) Same approach as between and within estimators, but we turn on both parameters this time
TWFE_mod = PanelOLS(
    df[y_var],
    df[X_vars],
    entity_effects=True,
    time_effects=True
).fit(cov_type="robust")

# ///////////////////////////
# First Differences Estimator
# ///////////////////////////
# We have to take NaN out, otherwise the run crashes
# (!!!) Faster to go directly with .diff() instead of filtering the already transformed variables from final_trans_df
# (!!!) I cannot call the df starting with a number like 1diff, like I would, cuz it's not accepted 
diff_df = df.groupby(level=0).diff().dropna()
# We now plug as usual 1diff in the PanelOLS
diff_mod = PanelOLS(
    diff_df[y_var],
    diff_df[X_vars]
).fit(cov_type="robust")

# ////////////
# Tabular Form
# ////////////
# (!!!) Nice comparison command (FANTASTIC!)
results = {
    "Between": between_mod,
    "OWFE-Within": within_mod,
    "RE-Mundlak": mundlak_mod,
    "TWFE-Within": TWFE_mod,
    "First Differences": diff_mod
}

comparison = compare(results)
comparison

,Between,OWFE-Within,RE-Mundlak,TWFE-Within,First Differences
Dep. Variable,GDP,GDP,GDP,GDP,GDP
Estimator,PanelOLS,PanelOLS,RandomEffects,PanelOLS,PanelOLS
No. Observations,731,731,731,731,555
Cov. Est.,Robust,Robust,Robust,Robust,Robust
R-squared,0.9521,0.9274,0.9353,0.9095,0.5109
R-Squared (Within),0.7718,0.9274,0.9274,0.9113,-0.1336
R-Squared (Between),0.9842,0.0219,0.9935,-0.1568,0.8463
R-Squared (Overall),0.9521,0.1464,0.9830,-0.0087,0.5109
F-statistic,2884.6,1816.1,1041.0,1330.2,114.90
P-value (F-stat),0.0000,0.0000,0.0000,0.0000,0.0000


##### <span style="color: orange"> **2.21) If, for the first differences dependent variable, it remains a simple auto-correlation above 0.1, a dynamic panel estimator can be tried. The estimators of the generalized method of moments (GMM) for panel data are only valid for short time panel T<10 and they face the issue of too many weak instruments. We suggest using its precursor, the Anderson-Hsiao (1981) estimator which allows to check the first stage of instrumental variables and to test for weak lagged instruments. Estimate an auto-regressive distributed lag (ARDL) model for dynamic panel data including the first lag of the dependent variable (for example: GDP per head growth) and the first lag of the key explanatory variable (for example: foreign aid/GDP), adding the first lag of other control variables is optional: Δ GDPGi,t = βy Δ GDPGi,t-1 + β1 Δ (aid/GDP)i,t + β2 Δ (aid/GDP)i,t-1 + Δ Controls i,t + Δ αi + Δ αt + Δ εi** </span>

### **2) PART 2B - OPTIONAL (if one of your variable is time-invariant z(i))**

[Unfortunately,  Lofaro, A., & Di Bucchianico, S. (2025) present PVAR model including 14 variables, country (i) and year (t) dependent, with only one exception for Energy Commodities Price Index, which is country-invariant, and hence only time dependent.]

##### <span style="color: orange"> **2.22) If one of your variable is time-invariant z(i), run a baseline Hausman Taylor estimation (pre-coded only in STATA) including all X(i.) as instruments. Comment the results. Else skip this question.** </span>

[n.a.]

##### <span style="color: orange"> **2.23) If one of your variable is time-invariant z(i), run a between regression on z(i) explained by X(i.) and other time invariant variable (only with N observations). If the R2 is low, this may signal X(i.) are weak instruments poorly correlated with the variable z(i) to be instrumented. Comment. Else skip this question.** </span>

[n.a.]

##### <span style="color: orange"> **2.24) If one of your variable is time-invariant z(i), as seen above, time invariant explanatory variables cannot explain the time varying within variance of the dependent variable and the Hausman Taylor internal instruments estimator is not so practical. Therefore, a practical shortcut is to include a time invariant variable multiplied by a time varying variable (interaction term): z(i) multiplied for x(it). Generate such a variable Include this product AND foreign aid into a one way fixed effects regression. Plot the estimated marginal effect (derivative) with respect to ICRG as a function of EDA/GDP (which is positive and goes as far as 20%).** </span>

[n.a.]